In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')
#%cd ..

from utils.data_extraction import load_data
from utils.data_cleaning import HFDataCleaning
#from .utils.data_extraction import load_data
#from .utils.data_cleaning import HFDataCleaning

### Load data etc

In [2]:
# print(os.listdir())
try:
    path = 'a:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
except:
    path = 't:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,32)]).astype(int)
dates = np.array(['20200401','20200402']).astype(int)

# Provide a list of tickers of interest
tickers = ['GOOG']#'MSFT'

# Do we need data on trades, quotes or both?
dataNeeded = 'trades' # 'trades', 'quotes' or 'both'

# run load_data()
if dataNeeded == 'trades':
    tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'quotes':
    quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'both':
    tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

['.ipynb_checkpoints', 'preprocessing.ipynb', 'testing_and_comparing_candlesticks.ipynb', 'testing_features.ipynb', 'testing_preprocessing_features.ipynb']
8515 ['taq_19930315.h5', 'taq_19930104.h5', 'taq_19930317.h5', 'taq_19930105.h5', 'taq_19930316.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200401
Date, Max: 20200402

##### Data Extraction begins #####

trade data is being extracted..

### Trade Data ###

The raw H5 trade file contains:  ['TradeIndex', 'Trades'] 

Ticker Information:  (b'GOOG            ', 26900500, 71427) 

Sneak peak of the data

            utcsec ex  cond  volume    price TradeStopStockIndicator corr  \
0  1

In [6]:
cleanedData = HFDataCleaning(['P1','p2','t1','p3'],tradeData,'trade',['q'])

In [102]:
step = 1#/10 # in minutes

aggregateMinute = np.arange(0,60,step)
aggregateHour = np.arange(9,16,1)
aggregateDate = np.arange(len(cleanedData.Date.unique()))

remove = 30//step

# candle = np.zeros(((len(aggregateDate)*len(aggregateMinute)*len(aggregateHour)),4))
candleNP = np.zeros((((len(aggregateDate)*len(aggregateMinute)*len(aggregateHour))-int(remove*len(aggregateDate))),4))

In [103]:
len(aggregateDate)*len(aggregateMinute)*len(aggregateHour)

840

In [104]:
remove*len(aggregateDate)

60

In [10]:
numpiedData = cleanedData[['Date','Hour','Minute']].to_numpy()
numpiedData = numpiedData.T
numpiedPrice = cleanedData['price'].to_numpy()

### Original

In [11]:


def candleCreateNP():
    ii = 0
    for l in cleanedData.Date.unique():
        for i in aggregateHour:
            for j in aggregateMinute:
                if (i == 9) & (j <30):
                    continue
                
                p1 = numpiedPrice[((numpiedData[0]==l)&\
                                     (numpiedData[1]==i)&\
                                     (numpiedData[2]>=j))&((numpiedData[0]==l)&\
                                                           (numpiedData[1]==i)&\
                                                           (numpiedData[2]<j+step))]
                if len(p1) > 0:
                    candleNP[ii] = np.array([p1[0],p1.max(),p1.min(),p1[-1]])
                else:
                    # if no new prices in the interval considered, use the previous pne
                    candleNP[ii] = candleNP[ii-1]
                ii += 1
                
    return candleNP

In [105]:
# Aggregate data in candle sticks
candleNP = candleCreateNP()

candleNP.shape

(780, 4)

In [13]:
candleNP[:5], candleNP[-5:]

(array([[1122.26, 1127.22, 1119.51, 1126.12],
        [1127.57, 1128.  , 1120.  , 1126.92],
        [1127.19, 1127.4 , 1124.02, 1125.44],
        [1125.01, 1125.19, 1122.97, 1123.  ],
        [1123.67, 1123.68, 1120.01, 1121.21]]),
 array([[1119.88 , 1122.88 , 1117.   , 1117.24 ],
        [1116.45 , 1122.96 , 1112.18 , 1121.41 ],
        [1121.43 , 1122.86 , 1119.76 , 1121.95 ],
        [1122.58 , 1123.6  , 1121.18 , 1122.86 ],
        [1122.725, 1124.   , 1119.64 , 1121.74 ]]))

### Attempt to vectorize

#### 26-06-2020: Get hour_min_col to include the full clock

In [203]:
cleanedData['Timestamp'][0].hour \
+ cleanedData['Timestamp'][0].minute/60 \
+ cleanedData['Timestamp'][0].second/(60*60) \
+ cleanedData['Timestamp'][0].microsecond/(60*60*10**6)

9.500003529444445

In [211]:
Timestamp_dt = cleanedData['Timestamp'].dt
Timestamp_float = Timestamp_dt.hour \
                  + Timestamp_dt.minute/60 \
                  + Timestamp_dt.second/(60*60) \
                  + Timestamp_dt.microsecond/(60*60*10**6)
cleanedData['hour_min_col'] = Timestamp_float

In [310]:
cleanedData.iloc[0], cleanedData.iloc[3700]

(utcsec                                 34200012706854
 ex                                                  Q
 cond                                             @ TI
 volume                                             83
 price                                         1122.26
 TradeStopStockIndicator                              
 corr                                               00
 TradeSequenceNumber                             36579
 TradeID                                           447
 SourceOfTrade                                       N
 TradeReportingFacility                               
 ParticipantTime                        34200012683939
 TRFTime                                            99
 TTE                                                 0
 Date                                         20200401
 Timestamp                  2020-04-01 09:30:00.012706
 TSRemainder                                       854
 Hour                                                9
 Minute   

In [212]:
#cleanedData['hour_min_col'] = cleanedData['Hour'] + cleanedData['Minute']/60
cleanedData.head()

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,...,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,hour_min_col,time_group
0,34200012706854,Q,@ TI,83,1122.26,,00,36579,447,N,...,99,0,20200401,2020-04-01 09:30:00.012706,854,9,30,GOOG,9.500004,0
1,34200012711048,Q,@ TI,17,1122.26,,00,36580,448,N,...,99,0,20200401,2020-04-01 09:30:00.012711,048,9,30,GOOG,9.500004,0
2,34200012771200,Q,@ TI,83,1122.26,,00,36583,449,N,...,99,0,20200401,2020-04-01 09:30:00.012771,200,9,30,GOOG,9.500004,0
3,34200012827903,Q,@ TI,83,1122.26,,00,36584,450,N,...,99,0,20200401,2020-04-01 09:30:00.012827,903,9,30,GOOG,9.500004,0
4,34200013120215,Q,@ TI,20,1122.08,,00,36589,451,N,...,99,0,20200401,2020-04-01 09:30:00.013120,215,9,30,GOOG,9.500004,0


In [221]:
31/60

0.5166666666666667

In [223]:
cleanedData['hour_min_col'][(cleanedData['hour_min_col'] > 9.5) & (cleanedData['hour_min_col'] < 16)]#9.5167

0         9.500004
1         9.500004
2         9.500004
3         9.500004
4         9.500004
5         9.500037
6         9.500038
7         9.500054
8         9.500054
9         9.500054
10        9.500054
11        9.500054
12        9.500130
13        9.500200
14        9.500200
15        9.500201
16        9.500241
17        9.500241
18        9.500241
19        9.500244
20        9.500271
21        9.500271
22        9.500281
23        9.500281
24        9.500285
25        9.500285
26        9.500296
27        9.500296
28        9.500306
29        9.500331
           ...    
42427    15.999815
42428    15.999836
42429    15.999845
42430    15.999871
42431    15.999876
42432    15.999876
42433    15.999876
42434    15.999876
42435    15.999876
42436    15.999876
42437    15.999876
42438    15.999876
42439    15.999876
42440    15.999890
42441    15.999890
42442    15.999890
42443    15.999890
42444    15.999890
42445    15.999890
42446    15.999921
42447    15.999951
42448    15.

In [225]:
cleanedData['hour_min_col'].min(), cleanedData['hour_min_col'].max()

(9.500003529444445, 15.999999836944443)

In [226]:
time_bins = np.arange(9.5-step/60,16+step/60,step/60)
time_bins, len(time_bins) 

(array([ 9.48333333,  9.5       ,  9.51666667,  9.53333333,  9.55      ,
         9.56666667,  9.58333333,  9.6       ,  9.61666667,  9.63333333,
         9.65      ,  9.66666667,  9.68333333,  9.7       ,  9.71666667,
         9.73333333,  9.75      ,  9.76666667,  9.78333333,  9.8       ,
         9.81666667,  9.83333333,  9.85      ,  9.86666667,  9.88333333,
         9.9       ,  9.91666667,  9.93333333,  9.95      ,  9.96666667,
         9.98333333, 10.        , 10.01666667, 10.03333333, 10.05      ,
        10.06666667, 10.08333333, 10.1       , 10.11666667, 10.13333333,
        10.15      , 10.16666667, 10.18333333, 10.2       , 10.21666667,
        10.23333333, 10.25      , 10.26666667, 10.28333333, 10.3       ,
        10.31666667, 10.33333333, 10.35      , 10.36666667, 10.38333333,
        10.4       , 10.41666667, 10.43333333, 10.45      , 10.46666667,
        10.48333333, 10.5       , 10.51666667, 10.53333333, 10.55      ,
        10.56666667, 10.58333333, 10.6       , 10.6

In [227]:
0.9833333333333333333*60

59.0

In [20]:
test_hour_min_col = pd.DataFrame([2,4,5,6,9])
test_hour_min_col
bins = [1,2,3,4,5,6,7,8,9,10]
iD = pd.cut(test_hour_min_col[0], bins=bins, right=True, labels=False)
iD
#--> so cut() actually properly skips a bin integer if there is no data in it

0    0
1    2
2    3
3    4
4    7
Name: 0, dtype: int64

In [21]:
test_df = pd.DataFrame({'content':np.nan},index=np.arange(len(bins)-1))
test_df

,content
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN


In [22]:
# if idD[-1] != iD.to_numpy()

In [23]:
# iD.to_numpy()[1:]-iD.to_numpy()[0:-1]

In [24]:
# test_df

In [25]:
test_df.iloc[iD.to_list(),0] = np.random.randint(1,10,iD.shape[0])

In [26]:
test_df

,content
0,6.0
1,NaN
2,2.0
3,4.0
4,9.0
5,NaN
6,NaN
7,8.0
8,NaN


In [27]:
test_df.fillna(method='ffill',inplace=True)

In [28]:
test_df

,content
0,6.0
1,6.0
2,2.0
3,4.0
4,9.0
5,9.0
6,9.0
7,8.0
8,8.0


In [29]:
emptyID = test_df[test_df.content==0].index.to_list()
toUse = list(map(lambda x: x - 1,emptyID))

In [30]:
toUse

[]

In [31]:
test_df.loc[toUse].content

Series([], Name: content, dtype: float64)

In [32]:
test_df.loc[toUse]

,content


In [33]:
toUse

[]

In [34]:
test_df.loc[toUse].content.to_numpy()

array([], dtype=float64)

In [35]:
emptyID

[]

In [36]:
test_df.loc[[3,6],'content']

3    4.0
6    9.0
Name: content, dtype: float64

In [37]:
test_df.loc[emptyID][['content']]

,content


In [230]:
# Does not work
test_df.loc[emptyID][['content']] = test_df.loc[toUse]

In [229]:
# Does not work
test_df.loc[emptyID,['content']] = test_df.loc[toUse]

In [228]:
# Works
test_df.loc[emptyID,'content'] = test_df.loc[toUse].content.to_list()

In [41]:
test_df.loc[toUse]

,content


In [42]:
test_df

,content
0,6.0
1,6.0
2,2.0
3,4.0
4,9.0
5,9.0
6,9.0
7,8.0
8,8.0


In [43]:
# Idea

# We create an empty dataframe, to hold all the candles.
test_df = pd.DataFrame({'content':0},index=bins)
# Our test hour dataframe
test_hour_min_col = pd.DataFrame([1,2,4,5])
# We use generate the time groups
bins = [1,2,3,4,5,6]
iD = pd.cut(test_hour_min_col[0], bins=bins, right=False, labels=False)

# We fill the vectorized-generated candles into the positions where there were trades.
test_df.iloc[iD.to_list(),0] = np.random.randint(1,10,iD.shape[0])

# We can know locate the empty indices
emptyID = test_df[test_df.content==0].index.to_list()

# We can use those indices to pass in the previous candles.
toUse = list(map(lambda x: x - 1,emptyID))

# We can now pass in the previous candles
test_df.loc[emptyID,'content'] = test_df.loc[toUse].content.to_list()
# The drawback of above, at the moment, is that the above only works for one series (Either Open, High, Low, Close)
# and not all of them. I have tried different options but without out luck for now.

In [243]:
step = 1
delta = step/60
time_bins = np.arange(9.5, 16+delta, delta) #np.arange(9.5-delta, 16+delta, delta)

In [244]:
time_bins.size

391

In [245]:
np.isin([0,1,2,3,4], pd.cut(test_hour_min_col[0], bins=[1,2,3,4,5,6], right=False, labels=False).values)

array([ True,  True, False,  True,  True])

In [246]:
np.arange(len(time_bins[1:-1]))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [247]:
cut_test = pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False).values
cut_test = cut_test[cut_test != 2]
cut_test

array([  0,   0,   0, ..., 389, 389, 389], dtype=int64)

In [248]:
#np.isin(np.arange(len(time_bins[1:-1])), pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False).values)
isin_cut = np.isin(np.arange(len(time_bins[1:-1])), cut_test)

np.where(isin_cut==False)

# create temporary version of cleanedData and add new observation with date, time_group, t-1 price for each missing. Sweep this in a while loop


(array([2], dtype=int64),)

In [249]:
cleanedData['hour_min_col']

0         9.500004
1         9.500004
2         9.500004
3         9.500004
4         9.500004
5         9.500037
6         9.500038
7         9.500054
8         9.500054
9         9.500054
10        9.500054
11        9.500054
12        9.500130
13        9.500200
14        9.500200
15        9.500201
16        9.500241
17        9.500241
18        9.500241
19        9.500244
20        9.500271
21        9.500271
22        9.500281
23        9.500281
24        9.500285
25        9.500285
26        9.500296
27        9.500296
28        9.500306
29        9.500331
           ...    
42427    15.999815
42428    15.999836
42429    15.999845
42430    15.999871
42431    15.999876
42432    15.999876
42433    15.999876
42434    15.999876
42435    15.999876
42436    15.999876
42437    15.999876
42438    15.999876
42439    15.999876
42440    15.999890
42441    15.999890
42442    15.999890
42443    15.999890
42444    15.999890
42445    15.999890
42446    15.999921
42447    15.999951
42448    15.

In [252]:
cleanedData['time_group'] = pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False)
cleanedData['time_group']

0          0
1          0
2          0
3          0
4          0
5          0
6          0
7          0
8          0
9          0
10         0
11         0
12         0
13         0
14         0
15         0
16         0
17         0
18         0
19         0
20         0
21         0
22         0
23         0
24         0
25         0
26         0
27         0
28         0
29         0
        ... 
42427    389
42428    389
42429    389
42430    389
42431    389
42432    389
42433    389
42434    389
42435    389
42436    389
42437    389
42438    389
42439    389
42440    389
42441    389
42442    389
42443    389
42444    389
42445    389
42446    389
42447    389
42448    389
42449    389
42450    389
42451    389
42452    389
42453    389
42454    389
42455    389
42456    389
Name: time_group, Length: 42457, dtype: int64

In [256]:
cleanedData.groupby(['Date','time_group'])[['price','Timestamp']].agg(['first', 'max', 'min', 'last'])

price                             \
                        first      max      min     last   
Date     time_group                                        
20200401 0           1122.260  1127.22  1119.51  1126.12   
         1           1127.570  1128.00  1120.00  1126.92   
         2           1127.190  1127.40  1124.02  1125.44   
         3           1125.010  1125.19  1122.97  1123.00   
         4           1123.670  1123.68  1120.01  1121.21   
         5           1121.270  1121.85  1114.79  1117.44   
         6           1117.040  1124.40  1117.04  1124.40   
         7           1124.980  1127.99  1124.97  1127.65   
         8           1127.630  1128.70  1125.18  1125.51   
         9           1126.950  1127.52  1124.27  1124.52   
         10          1124.720  1125.89  1123.07  1124.52   
         11          1124.120  1126.00  1123.80  1125.77   
         12          1125.810  1127.62  1125.35  1125.35   
         13          1126.550  1127.85  1125.18  1125.18   
         14          1125.830  1126.47  1124.68  1124.76   
         15          1124.140  1124.98  1122.50  1122.50   
         16          1122.930  1122.93  1118.26  1118.26   
         17          1119.690  1120.80  1118.88  1119.07   
         18          1119.560  1120.63  1119.00  1119.77   
         19          1120.000  1121.81  1119.73  1120.29   
         20          1120.130  1121.52  1119.70  1120.75   
         21          1121.680  1121.68  1119.98  1120.19   
         22          1120.190  1120.30  1117.34  1117.61   
         23          1118.670  1120.93  1118.03  1120.93   
         24          1121.310  1121.61  1118.50  1118.92   
         25          1118.050  1118.92  1117.70  1118.57   
         26          1118.710  1119.76  1117.60  1118.55   
         27          1118.750  1118.80  1116.46  1118.66   
         28          1118.120  1118.12  1115.30  1115.30   
         29          1115.500  1116.98  1115.18  1115.55   
...                       ...      ...      ...      ...   
20200402 360         1114.860  1117.09  1114.86  1115.41   
         361         1114.600  1114.60  1110.56  1112.52   
         362         1112.840  1114.34  1112.83  1112.83   
         363         1113.320  1114.20  1112.47  1113.80   
         364         1113.300  1114.20  1112.36  1112.52   
         365         1112.500  1114.57  1112.50  1113.45   
         366         1113.820  1115.44  1112.35  1112.35   
         367         1112.360  1112.36  1110.43  1111.08   
         368         1111.580  1112.42  1111.13  1112.21   
         369         1111.610  1113.34  1111.61  1112.42   
         370         1112.700  1115.34  1112.70  1114.36   
         371         1114.360  1116.00  1112.71  1115.06   
         372         1115.670  1117.39  1113.93  1114.38   
         373         1113.940  1117.46  1113.50  1115.60   
         374         1115.070  1118.00  1115.07  1117.09   
         375         1116.560  1118.60  1116.23  1118.21   
         376         1117.690  1117.69  1115.00  1115.28   
         377         1115.310  1116.41  1113.00  1113.50   
         378         1113.770  1114.24  1112.79  1113.53   
         379         1113.790  1114.12  1111.00  1114.12   
         380         1114.940  1121.08  1114.94  1119.84   
         381         1120.400  1120.40  1116.59  1117.08   
         382         1117.290  1118.25  1115.66  1116.62   
         383         1117.050  1118.24  1114.64  1114.80   
         384         1115.870  1123.67  1115.72  1120.98   
         385         1119.880  1122.88  1117.00  1117.24   
         386         1116.450  1122.96  1112.18  1121.41   
         387         1121.430  1122.86  1119.76  1121.95   
         388         1122.580  1123.60  1121.18  1122.86   
         389         1122.725  1124.00  1119.64  1121.74   

                                     Timestamp                             \
                                         first                        max   
Date     time_group          

In [257]:
cleanedData[test2==0]

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,...,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,hour_min_col,time_group
0,34200012706854,Q,@ TI,83,1122.26,,00,36579,447,N,...,99,0,20200401,2020-04-01 09:30:00.012706,854,9,30,GOOG,9.500004,0
1,34200012711048,Q,@ TI,17,1122.26,,00,36580,448,N,...,99,0,20200401,2020-04-01 09:30:00.012711,048,9,30,GOOG,9.500004,0
2,34200012771200,Q,@ TI,83,1122.26,,00,36583,449,N,...,99,0,20200401,2020-04-01 09:30:00.012771,200,9,30,GOOG,9.500004,0
3,34200012827903,Q,@ TI,83,1122.26,,00,36584,450,N,...,99,0,20200401,2020-04-01 09:30:00.012827,903,9,30,GOOG,9.500004,0
4,34200013120215,Q,@ TI,20,1122.08,,00,36589,451,N,...,99,0,20200401,2020-04-01 09:30:00.013120,215,9,30,GOOG,9.500004,0
5,34200134121465,Q,@ TI,1,1122.25,,00,36816,452,N,...,99,0,20200401,2020-04-01 09:30:00.134121,465,9,30,GOOG,9.500037,0
6,34200137711405,Q,@ TI,1,1121.00,,00,36821,453,N,...,99,0,20200401,2020-04-01 09:30:00.137711,405,9,30,GOOG,9.500038,0
7,34200194838863,Q,@FTI,38,1122.26,,00,36870,454,N,...,99,1,20200401,2020-04-01 09:30:00.194838,863,9,30,GOOG,9.500054,0
8,34200194842233,Q,@FTI,1,1122.26,,00,36871,455,N,...,99,1,20200401,2020-04-01 09:30:00.194842,233,9,30,GOOG,9.500054,0
9,34200194877651,Q,@FTI,15,1124.00,,00,36872,456,N,...,99,1,20200401,2020-04-01 09:30:00.194877,651,9,30,GOOG,9.500054,0


In [258]:
test2[test2 ==0]

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
23673    0
23674    0
23675    0
23676    0
23677    0
23678    0
23679    0
23680    0
23681    0
23682    0
23683    0
23684    0
23685    0
23686    0
23687    0
23688    0
23689    0
23690    0
23691    0
23692    0
23693    0
23694    0
23695    0
23696    0
23697    0
23698    0
23699    0
23700    0
23701    0
23702    0
Name: hour_min_col, Length: 919, dtype: int64

In [262]:
cleanedData['time_group'] = pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False) #0#+1
cleanedData

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,...,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,hour_min_col,time_group
0,34200012706854,Q,@ TI,83,1122.260,,00,36579,447,N,...,99,0,20200401,2020-04-01 09:30:00.012706,854,9,30,GOOG,9.500004,0
1,34200012711048,Q,@ TI,17,1122.260,,00,36580,448,N,...,99,0,20200401,2020-04-01 09:30:00.012711,048,9,30,GOOG,9.500004,0
2,34200012771200,Q,@ TI,83,1122.260,,00,36583,449,N,...,99,0,20200401,2020-04-01 09:30:00.012771,200,9,30,GOOG,9.500004,0
3,34200012827903,Q,@ TI,83,1122.260,,00,36584,450,N,...,99,0,20200401,2020-04-01 09:30:00.012827,903,9,30,GOOG,9.500004,0
4,34200013120215,Q,@ TI,20,1122.080,,00,36589,451,N,...,99,0,20200401,2020-04-01 09:30:00.013120,215,9,30,GOOG,9.500004,0
5,34200134121465,Q,@ TI,1,1122.250,,00,36816,452,N,...,99,0,20200401,2020-04-01 09:30:00.134121,465,9,30,GOOG,9.500037,0
6,34200137711405,Q,@ TI,1,1121.000,,00,36821,453,N,...,99,0,20200401,2020-04-01 09:30:00.137711,405,9,30,GOOG,9.500038,0
7,34200194838863,Q,@FTI,38,1122.260,,00,36870,454,N,...,99,1,20200401,2020-04-01 09:30:00.194838,863,9,30,GOOG,9.500054,0
8,34200194842233,Q,@FTI,1,1122.260,,00,36871,455,N,...,99,1,20200401,2020-04-01 09:30:00.194842,233,9,30,GOOG,9.500054,0
9,34200194877651,Q,@FTI,15,1124.000,,00,36872,456,N,...,99,1,20200401,2020-04-01 09:30:00.194877,651,9,30,GOOG,9.500054,0


In [259]:
# check for missing time_group
np.unique(cleanedData['time_group'])

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [260]:
cleanedData[(cleanedData['hour_min_col'] > 9.6) & (cleanedData['hour_min_col'] < 9.7)]

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,...,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,hour_min_col,time_group
1578,34560055668436,Q,@ I,2,1117.04,,00,140261,2025,N,...,99,0,20200401,2020-04-01 09:36:00.055668,436,9,36,GOOG,9.600015,6
1579,34560217790453,Q,@F I,2,1117.05,,00,140987,2026,N,...,99,1,20200401,2020-04-01 09:36:00.217790,453,9,36,GOOG,9.600060,6
1580,34561110995117,Q,@ I,25,1117.14,,00,141266,2027,N,...,99,0,20200401,2020-04-01 09:36:01.110995,117,9,36,GOOG,9.600309,6
1581,34561116822206,Q,@F I,50,1117.13,,00,141271,2028,N,...,99,1,20200401,2020-04-01 09:36:01.116822,206,9,36,GOOG,9.600310,6
1582,34561116872911,Q,@ I,25,1117.13,,00,141272,2029,N,...,99,0,20200401,2020-04-01 09:36:01.116872,911,9,36,GOOG,9.600310,6
1583,34563723490500,Q,@ I,34,1117.92,,00,141909,2030,N,...,99,0,20200401,2020-04-01 09:36:03.723490,500,9,36,GOOG,9.601034,6
1584,34563723601722,Q,@ I,10,1117.92,,00,141912,2031,N,...,99,0,20200401,2020-04-01 09:36:03.723601,722,9,36,GOOG,9.601034,6
1585,34563974203574,Q,@F,100,1117.96,,00,141939,2032,N,...,99,1,20200401,2020-04-01 09:36:03.974203,574,9,36,GOOG,9.601104,6
1586,34563974207654,Q,@F I,9,1117.96,,00,141940,2033,N,...,99,1,20200401,2020-04-01 09:36:03.974207,654,9,36,GOOG,9.601104,6
1587,34563974219094,Q,@F I,13,1117.96,,00,141941,2034,N,...,99,1,20200401,2020-04-01 09:36:03.974219,094,9,36,GOOG,9.601104,6


In [261]:
cleanedData.groupby(['Date','time_group'])[['price','Timestamp']].last() # .last()

# or insert new observation here with the missing time_group

price                  Timestamp
Date     time_group                                    
20200401 0           1126.12 2020-04-01 09:30:58.001385
         1           1126.92 2020-04-01 09:31:59.029981
         2           1125.44 2020-04-01 09:32:59.161537
         3           1123.00 2020-04-01 09:33:54.125338
         4           1121.21 2020-04-01 09:34:58.749776
         5           1117.44 2020-04-01 09:35:57.004849
         6           1124.40 2020-04-01 09:36:59.589734
         7           1127.65 2020-04-01 09:37:58.782698
         8           1125.51 2020-04-01 09:38:57.695308
         9           1124.52 2020-04-01 09:39:56.779269
         10          1124.52 2020-04-01 09:40:53.347684
         11          1125.77 2020-04-01 09:41:55.758752
         12          1125.35 2020-04-01 09:42:57.084996
         13          1125.18 2020-04-01 09:43:57.015881
         14          1124.76 2020-04-01 09:44:53.256203
         15          1122.50 2020-04-01 09:45:53.315858
         16          1118.26 2020-04-01 09:46:49.183536
         17          1119.07 2020-04-01 09:47:52.330064
         18          1119.77 2020-04-01 09:48:44.594046
         19          1120.29 2020-04-01 09:49:58.874800
         20          1120.75 2020-04-01 09:50:53.389917
         21          1120.19 2020-04-01 09:51:55.484069
         22          1117.61 2020-04-01 09:52:56.637265
         23          1120.93 2020-04-01 09:53:57.644114
         24          1118.92 2020-04-01 09:54:58.468796
         25          1118.57 2020-04-01 09:55:50.936623
         26          1118.55 2020-04-01 09:56:54.750055
         27          1118.66 2020-04-01 09:57:56.922049
         28          1115.30 2020-04-01 09:58:59.858874
         29          1115.55 2020-04-01 09:59:59.939460
...                      ...                        ...
20200402 360         1115.41 2020-04-02 15:30:48.613364
         361         1112.52 2020-04-02 15:31:59.023634
         362         1112.83 2020-04-02 15:32:40.372046
         363         1113.80 2020-04-02 15:33:58.119901
         364         1112.52 2020-04-02 15:34:55.170432
         365         1113.45 2020-04-02 15:35:57.973405
         366         1112.35 2020-04-02 15:36:53.861227
         367         1111.08 2020-04-02 15:37:57.683002
         368         1112.21 2020-04-02 15:38:55.066361
         369         1112.42 2020-04-02 15:39:47.617796
         370         1114.36 2020-04-02 15:40:55.080594
         371         1115.06 2020-04-02 15:41:59.530284
         372         1114.38 2020-04-02 15:42:48.901349
         373         1115.60 2020-04-02 15:43:47.341511
         374         1117.09 2020-04-02 15:44:49.800851
         375         1118.21 2020-04-02 15:45:58.374949
         376         1115.28 2020-04-02 15:46:59.970785
         377         1113.50 2020-04-02 15:47:57.025937
         378         1113.53 2020-04-02 15:48:59.231967
         379         1114.12 2020-04-02 15:49:59.176315
         380         1119.84 2020-04-02 15:50:57.258259
         381         1117.08 2020-04-02 15:51:59.546425
         382         1116.62 2020-04-02 15:52:56.482767
         383         1114.80 2020-04-02 15:53:59.856663
         384         1120.98 2020-04-02 15:54:59.405164
         385         1117.24 2020-04-02 15:55:59.755459
         386         1121.41 2020-04-02 15:56:59.071091
         387         1121.95 2020-04-02 15:57:58.132701
         388         1122.86 2020-04-02 15:58:59.458451
         389         1121.74 2020-04-02 15:59:59.999413

[780 rows x 2 columns]

In [58]:
OHLC = cleanedData.groupby(['Date','time_group'])[['price']].agg(['first', 'max', 'min', 'last'])
OHLC.values.shape

(780, 4)

In [59]:
OHLC.values[:5], OHLC.values[-5:]

(array([[1122.26, 1127.22, 1119.51, 1126.12],
        [1127.57, 1128.  , 1120.  , 1126.92],
        [1127.19, 1127.4 , 1124.02, 1125.44],
        [1125.01, 1125.19, 1122.97, 1123.  ],
        [1123.67, 1123.68, 1120.01, 1121.21]]),
 array([[1119.88 , 1122.88 , 1117.   , 1117.24 ],
        [1116.45 , 1122.96 , 1112.18 , 1121.41 ],
        [1121.43 , 1122.86 , 1119.76 , 1121.95 ],
        [1122.58 , 1123.6  , 1121.18 , 1122.86 ],
        [1122.725, 1124.   , 1119.64 , 1121.74 ]]))

In [60]:
# set up as function
def candleCreateNP_vect(verbose=True):
        
    cleanedData['hour_min_col'] = cleanedData['Hour'] + cleanedData['Minute']/60
    if verbose:
        print(f"min and max of new hour_min_col: \
              {cleanedData['hour_min_col'].min()}, {cleanedData['hour_min_col'].max()}")
              
    # setup time_bins to group each timestamp
    #print(step)
    delta = step/60
    #print(delta)
    time_bins = np.arange(9.5-delta, 16, delta)
    #print(len(time_bins))
              
    # put each timestamp into a bucket according to time_bins defined by the step variable
    cleanedData['time_group'] = pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False)
    
    # group by date and time_group, extract price, take it first, max, min, last (open, high, low, close)
    OHLC = cleanedData.groupby(['Date','time_group'])[['price']].agg(['first', 'max', 'min', 'last'])              
    
    # return as numpy if preferred
    return OHLC#.values

In [61]:
test_candles=candleCreateNP_vect()

min and max of new hour_min_col:               9.5, 15.983333333333333


In [62]:
test_candles

price                           
                        first      max      min     last
Date     time_group                                     
20200401 0           1122.260  1127.22  1119.51  1126.12
         1           1127.570  1128.00  1120.00  1126.92
         2           1127.190  1127.40  1124.02  1125.44
         3           1125.010  1125.19  1122.97  1123.00
         4           1123.670  1123.68  1120.01  1121.21
         5           1121.270  1121.85  1114.79  1117.44
         6           1117.040  1124.40  1117.04  1124.40
         7           1124.980  1127.99  1124.97  1127.65
         8           1127.630  1128.70  1125.18  1125.51
         9           1126.950  1127.52  1124.27  1124.52
         10          1124.720  1125.89  1123.07  1124.52
         11          1124.120  1126.00  1123.80  1125.77
         12          1125.810  1127.62  1125.35  1125.35
         13          1126.550  1127.85  1125.18  1125.18
         14          1125.830  1126.47  1124.68  1124.76
         15          1124.140  1124.98  1122.50  1122.50
         16          1122.930  1122.93  1118.26  1118.26
         17          1119.690  1120.80  1118.88  1119.07
         18          1119.560  1120.63  1119.00  1119.77
         19          1120.000  1121.81  1119.73  1120.29
         20          1120.130  1121.52  1119.70  1120.75
         21          1121.680  1121.68  1119.98  1120.19
         22          1120.190  1120.30  1117.34  1117.61
         23          1118.670  1120.93  1118.03  1120.93
         24          1121.310  1121.61  1118.50  1118.92
         25          1118.050  1118.92  1117.70  1118.57
         26          1118.710  1119.76  1117.60  1118.55
         27          1118.750  1118.80  1116.46  1118.66
         28          1118.120  1118.12  1115.30  1115.30
         29          1115.500  1116.98  1115.18  1115.55
...                       ...      ...      ...      ...
20200402 360         1114.860  1117.09  1114.86  1115.41
         361         1114.600  1114.60  1110.56  1112.52
         362         1112.840  1114.34  1112.83  1112.83
         363         1113.320  1114.20  1112.47  1113.80
         364         1113.300  1114.20  1112.36  1112.52
         365         1112.500  1114.57  1112.50  1113.45
         366         1113.820  1115.44  1112.35  1112.35
         367         1112.360  1112.36  1110.43  1111.08
         368         1111.580  1112.42  1111.13  1112.21
         369         1111.610  1113.34  1111.61  1112.42
         370         1112.700  1115.34  1112.70  1114.36
         371         1114.360  1116.00  1112.71  1115.06
         372         1115.670  1117.39  1113.93  1114.38
         373         1113.940  1117.46  1113.50  1115.60
         374         1115.070  1118.00  1115.07  1117.09
         375         1116.560  1118.60  1116.23  1118.21
         376         1117.690  1117.69  1115.00  1115.28
         377         1115.310  1116.41  1113.00  1113.50
         378         1113.770  1114.24  1112.79  1113.53
         379         1113.790  1114.12  1111.00  1114.12
         380         1114.940  1121.08  1114.94  1119.84
         381         1120.400  1120.40  1116.59  1117.08
         382         1117.290  1118.25  1115.66  1116.62
         383         1117.050  1118.24  1114.64  1114.80
         384         1115.870  1123.67  1115.72  1120.98
         385         1119.880  1122.88  1117.00  1117.24
         386         1116.450  1122.96  1112.18  1121.41
         387         1121.430  1122.86  1119.76  1121.95
         388         1122.580  1123.60  1121.18  1122.86
         389         1122.725  1124.00  1119.64  1121.74

[780 rows x 4 columns]

### Compare timing once

In [63]:
import time
a = time.time()
candleNP1 = candleCreateNP()
b = time.time()
print(b-a)

a = time.time()
candleNP2 = candleCreateNP_vect()
b = time.time()
print(b-a)

4.280244827270508
min and max of new hour_min_col:               9.5, 15.983333333333333
0.019001007080078125


In [64]:
# Make sure they are equal
(candleNP1.shape == candleNP2.shape), \
(candleNP1 == candleNP2).all(), \
np.array_equal(candleNP1, candleNP2)

(True, price  first    True
        max      True
        min      True
        last     True
 dtype: bool, True)

### Multiple time-it runs

In [24]:
# Aggregate data in candle sticks
%timeit candleNP1 = candleCreateNP()
%timeit candleNP2 = candleCreateNP_vect(verbose=False)

56.8 s ± 498 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
132 ms ± 2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Let's build something awesome - something that than fill black holes with lightning speed

In [65]:
step = 1
delta = step/60
time_bins = np.arange(9.5-delta
                      ,16#+delta
                      ,delta)

In [66]:
testy = candleCreateNP_vect()

min and max of new hour_min_col:               9.5, 15.983333333333333


In [67]:
testy

price                           
                        first      max      min     last
Date     time_group                                     
20200401 0           1122.260  1127.22  1119.51  1126.12
         1           1127.570  1128.00  1120.00  1126.92
         2           1127.190  1127.40  1124.02  1125.44
         3           1125.010  1125.19  1122.97  1123.00
         4           1123.670  1123.68  1120.01  1121.21
         5           1121.270  1121.85  1114.79  1117.44
         6           1117.040  1124.40  1117.04  1124.40
         7           1124.980  1127.99  1124.97  1127.65
         8           1127.630  1128.70  1125.18  1125.51
         9           1126.950  1127.52  1124.27  1124.52
         10          1124.720  1125.89  1123.07  1124.52
         11          1124.120  1126.00  1123.80  1125.77
         12          1125.810  1127.62  1125.35  1125.35
         13          1126.550  1127.85  1125.18  1125.18
         14          1125.830  1126.47  1124.68  1124.76
         15          1124.140  1124.98  1122.50  1122.50
         16          1122.930  1122.93  1118.26  1118.26
         17          1119.690  1120.80  1118.88  1119.07
         18          1119.560  1120.63  1119.00  1119.77
         19          1120.000  1121.81  1119.73  1120.29
         20          1120.130  1121.52  1119.70  1120.75
         21          1121.680  1121.68  1119.98  1120.19
         22          1120.190  1120.30  1117.34  1117.61
         23          1118.670  1120.93  1118.03  1120.93
         24          1121.310  1121.61  1118.50  1118.92
         25          1118.050  1118.92  1117.70  1118.57
         26          1118.710  1119.76  1117.60  1118.55
         27          1118.750  1118.80  1116.46  1118.66
         28          1118.120  1118.12  1115.30  1115.30
         29          1115.500  1116.98  1115.18  1115.55
...                       ...      ...      ...      ...
20200402 360         1114.860  1117.09  1114.86  1115.41
         361         1114.600  1114.60  1110.56  1112.52
         362         1112.840  1114.34  1112.83  1112.83
         363         1113.320  1114.20  1112.47  1113.80
         364         1113.300  1114.20  1112.36  1112.52
         365         1112.500  1114.57  1112.50  1113.45
         366         1113.820  1115.44  1112.35  1112.35
         367         1112.360  1112.36  1110.43  1111.08
         368         1111.580  1112.42  1111.13  1112.21
         369         1111.610  1113.34  1111.61  1112.42
         370         1112.700  1115.34  1112.70  1114.36
         371         1114.360  1116.00  1112.71  1115.06
         372         1115.670  1117.39  1113.93  1114.38
         373         1113.940  1117.46  1113.50  1115.60
         374         1115.070  1118.00  1115.07  1117.09
         375         1116.560  1118.60  1116.23  1118.21
         376         1117.690  1117.69  1115.00  1115.28
         377         1115.310  1116.41  1113.00  1113.50
         378         1113.770  1114.24  1112.79  1113.53
         379         1113.790  1114.12  1111.00  1114.12
         380         1114.940  1121.08  1114.94  1119.84
         381         1120.400  1120.40  1116.59  1117.08
         382         1117.290  1118.25  1115.66  1116.62
         383         1117.050  1118.24  1114.64  1114.80
         384         1115.870  1123.67  1115.72  1120.98
         385         1119.880  1122.88  1117.00  1117.24
         386         1116.450  1122.96  1112.18  1121.41
         387         1121.430  1122.86  1119.76  1121.95
         388         1122.580  1123.60  1121.18  1122.86
         389         1122.725  1124.00  1119.64  1121.74

[780 rows x 4 columns]

In [68]:
np.random.seed(2019)
for dt in testy.index.get_level_values(0).unique():
    dropedElements = []
    print(dt,'\n')
    for inx in np.arange(10):
        rm = np.random.randint(0,390,1)
        #print(rm)
        dropedElements.append(rm[0])
        testy = testy.drop((dt,rm[0]))
    print('Elements dropped: ',dropedElements)

20200401 

Elements dropped:  [72, 370, 159, 293, 216, 190, 246, 24, 285, 271]
20200402 

Elements dropped:  [106, 16, 176, 199, 229, 83, 12, 178, 133, 375]


In [69]:
# Some elements was dropped
testy.index.get_level_values(1).shape

(760,)

In [70]:
##### Creating our temporary table, with all the indices that is surposed to be in the actual candle-table.
## Creating the multiIndex-index
mtInd = pd.MultiIndex.from_product([testy.index.get_level_values(0).unique(), np.arange(len(time_bins)-2)],
                           names=['Date','time_group'])

## Creating the multiIndex-columns
mtCol = pd.MultiIndex.from_product([['price'],['first','high','low','last']])

## Creating the table itself
tempDf = pd.DataFrame(np.nan
                      ,columns=mtCol
                      ,index=mtInd)

In [73]:
[testy.index.get_level_values(0).unique()]

[Index(['20200401', '20200402'], dtype='object', name='Date')]

In [77]:
[np.arange(len(time_bins)-2)]

[array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [78]:
mtCol

MultiIndex(levels=[['price'], ['first', 'high', 'last', 'low']],
           codes=[[0, 0, 0, 0], [0, 1, 3, 2]])

In [71]:
tempDf

price              
                    first high low last
Date     time_group                    
20200401 0            NaN  NaN NaN  NaN
         1            NaN  NaN NaN  NaN
         2            NaN  NaN NaN  NaN
         3            NaN  NaN NaN  NaN
         4            NaN  NaN NaN  NaN
         5            NaN  NaN NaN  NaN
         6            NaN  NaN NaN  NaN
         7            NaN  NaN NaN  NaN
         8            NaN  NaN NaN  NaN
         9            NaN  NaN NaN  NaN
         10           NaN  NaN NaN  NaN
         11           NaN  NaN NaN  NaN
         12           NaN  NaN NaN  NaN
         13           NaN  NaN NaN  NaN
         14           NaN  NaN NaN  NaN
         15           NaN  NaN NaN  NaN
         16           NaN  NaN NaN  NaN
         17           NaN  NaN NaN  NaN
         18           NaN  NaN NaN  NaN
         19           NaN  NaN NaN  NaN
         20           NaN  NaN NaN  NaN
         21           NaN  NaN NaN  NaN
         22           NaN  NaN NaN  NaN
         23           NaN  NaN NaN  NaN
         24           NaN  NaN NaN  NaN
         25           NaN  NaN NaN  NaN
         26           NaN  NaN NaN  NaN
         27           NaN  NaN NaN  NaN
         28           NaN  NaN NaN  NaN
         29           NaN  NaN NaN  NaN
...                   ...  ...  ..  ...
20200402 360          NaN  NaN NaN  NaN
         361          NaN  NaN NaN  NaN
         362          NaN  NaN NaN  NaN
         363          NaN  NaN NaN  NaN
         364          NaN  NaN NaN  NaN
         365          NaN  NaN NaN  NaN
         366          NaN  NaN NaN  NaN
         367          NaN  NaN NaN  NaN
         368          NaN  NaN NaN  NaN
         369          NaN  NaN NaN  NaN
         370          NaN  NaN NaN  NaN
         371          NaN  NaN NaN  NaN
         372          NaN  NaN NaN  NaN
         373          NaN  NaN NaN  NaN
         374          NaN  NaN NaN  NaN
         375          NaN  NaN NaN  NaN
         376          NaN  NaN NaN  NaN
         377          NaN  NaN NaN  NaN
         378          NaN  NaN NaN  NaN
         379          NaN  NaN NaN  NaN
         380          NaN  NaN NaN  NaN
         381          NaN  NaN NaN  NaN
         382          NaN  NaN NaN  NaN
         383          NaN  NaN NaN  NaN
         384          NaN  NaN NaN  NaN
         385          NaN  NaN NaN  NaN
         386          NaN  NaN NaN  NaN
         387          NaN  NaN NaN  NaN
         388          NaN  NaN NaN  NaN
         389          NaN  NaN NaN  NaN

[780 rows x 4 columns]

In [79]:
testy.shape, tempDf.shape

((760, 4), (780, 4))

In [81]:
tempDf.loc[testy.index]

price                           
                        first     high      low     last
Date     time_group                                     
20200401 0           1122.260  1127.22  1119.51  1126.12
         1           1127.570  1128.00  1120.00  1126.92
         2           1127.190  1127.40  1124.02  1125.44
         3           1125.010  1125.19  1122.97  1123.00
         4           1123.670  1123.68  1120.01  1121.21
         5           1121.270  1121.85  1114.79  1117.44
         6           1117.040  1124.40  1117.04  1124.40
         7           1124.980  1127.99  1124.97  1127.65
         8           1127.630  1128.70  1125.18  1125.51
         9           1126.950  1127.52  1124.27  1124.52
         10          1124.720  1125.89  1123.07  1124.52
         11          1124.120  1126.00  1123.80  1125.77
         12          1125.810  1127.62  1125.35  1125.35
         13          1126.550  1127.85  1125.18  1125.18
         14          1125.830  1126.47  1124.68  1124.76
         15          1124.140  1124.98  1122.50  1122.50
         16          1122.930  1122.93  1118.26  1118.26
         17          1119.690  1120.80  1118.88  1119.07
         18          1119.560  1120.63  1119.00  1119.77
         19          1120.000  1121.81  1119.73  1120.29
         20          1120.130  1121.52  1119.70  1120.75
         21          1121.680  1121.68  1119.98  1120.19
         22          1120.190  1120.30  1117.34  1117.61
         23          1118.670  1120.93  1118.03  1120.93
         25          1118.050  1118.92  1117.70  1118.57
         26          1118.710  1119.76  1117.60  1118.55
         27          1118.750  1118.80  1116.46  1118.66
         28          1118.120  1118.12  1115.30  1115.30
         29          1115.500  1116.98  1115.18  1115.55
         30          1115.350  1119.26  1115.35  1116.00
...                       ...      ...      ...      ...
20200402 359         1113.460  1114.65  1113.41  1114.60
         360         1114.860  1117.09  1114.86  1115.41
         361         1114.600  1114.60  1110.56  1112.52
         362         1112.840  1114.34  1112.83  1112.83
         363         1113.320  1114.20  1112.47  1113.80
         364         1113.300  1114.20  1112.36  1112.52
         365         1112.500  1114.57  1112.50  1113.45
         366         1113.820  1115.44  1112.35  1112.35
         367         1112.360  1112.36  1110.43  1111.08
         368         1111.580  1112.42  1111.13  1112.21
         369         1111.610  1113.34  1111.61  1112.42
         370         1112.700  1115.34  1112.70  1114.36
         371         1114.360  1116.00  1112.71  1115.06
         372         1115.670  1117.39  1113.93  1114.38
         373         1113.940  1117.46  1113.50  1115.60
         374         1115.070  1118.00  1115.07  1117.09
         376         1117.690  1117.69  1115.00  1115.28
         377         1115.310  1116.41  1113.00  1113.50
         378         1113.770  1114.24  1112.79  1113.53
         379         1113.790  1114.12  1111.00  1114.12
         380         1114.940  1121.08  1114.94  1119.84
         381         1120.400  1120.40  1116.59  1117.08
         382         1117.290  1118.25  1115.66  1116.62
         383         1117.050  1118.24  1114.64  1114.80
         384         1115.870  1123.67  1115.72  1120.98
         385         1119.880  1122.88  1117.00  1117.24
         386         1116.450  1122.96  1112.18  1121.41
         387         1121.430  1122.86  1119.76  1121.95
         388         1122.580  1123.60  1121.18  1122.86
         389         1122.725  1124.00  1119.64  1121.74

[760 rows x 4 columns]

In [80]:
# Filling the elements of testy into the temp-table - remember we just droped some elements, so we will get some nans
tempDf.loc[testy.index]=testy.copy(deep=True)

In [82]:
# Checking the nans in the temporary table - Voila! we have some to correct.
# It should not matter which "price" we check on, because if there is just one price within the time_group - there we be 
# values for all types of "prices" (first, high, low, last). 
tempDf[tempDf.price['first'].isna()].shape

(20, 4)

In [83]:
# Another way to see this.
tempDf.count()

price  first    760
       high     760
       low      760
       last     760
dtype: int64

In [85]:
# Storing the indices of the empty elements, to be corrected.
testIndex = tempDf[tempDf.price['first'].isna()].index
testIndex

MultiIndex(levels=[['20200401', '20200402'], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212,

In [126]:
tempDf.price['last'].fillna(method='ffill').loc[testIndex]

Date      time_group
20200401  24            1120.93
          72            1126.05
          159           1115.26
          190           1106.76
          216           1111.25
          246           1109.42
          271           1112.04
          285           1110.20
          293           1109.24
          370           1097.88
20200402  12            1105.00
          16            1096.50
          83            1103.50
          106           1109.71
          133           1114.61
          176           1119.50
          178           1118.71
          199           1122.25
          229           1114.71
          375           1117.09
Name: last, dtype: float64

In [228]:
# We can fill the empty elements with the previous non-empty value of the last price in the following way
pd.DataFrame({('price','first'):tempDf.price['last'].fillna(method='ffill').loc[testIndex],
              ('price','high'):tempDf.price['last'].fillna(method='ffill').loc[testIndex],
              ('price','low'):tempDf.price['last'].fillna(method='ffill').loc[testIndex],
              ('price','last'):tempDf.price['last'].fillna(method='ffill').loc[testIndex]})

price                           
                       first     high      low     last
Date     time_group                                    
20200401 24          1120.93  1120.93  1120.93  1120.93
         72          1126.05  1126.05  1126.05  1126.05
         159         1115.26  1115.26  1115.26  1115.26
         190         1106.76  1106.76  1106.76  1106.76
         216         1111.25  1111.25  1111.25  1111.25
         246         1109.42  1109.42  1109.42  1109.42
         271         1112.04  1112.04  1112.04  1112.04
         285         1110.20  1110.20  1110.20  1110.20
         293         1109.24  1109.24  1109.24  1109.24
         370         1097.88  1097.88  1097.88  1097.88
20200402 12          1105.00  1105.00  1105.00  1105.00
         16          1096.50  1096.50  1096.50  1096.50
         83          1103.50  1103.50  1103.50  1103.50
         106         1109.71  1109.71  1109.71  1109.71
         133         1114.61  1114.61  1114.61  1114.61
         176         1119.50  1119.50  1119.50  1119.50
         178         1118.71  1118.71  1118.71  1118.71
         199         1122.25  1122.25  1122.25  1122.25
         229         1114.71  1114.71  1114.71  1114.71
         375         1117.09  1117.09  1117.09  1117.09

In [86]:
def test_fill_values_1():
    # We can fill the empty elements with the previous non-empty value of the last price in the following way
    pd.DataFrame({('price','first'):tempDf.price['last'].fillna(method='ffill').loc[testIndex],
                  ('price','high'):tempDf.price['last'].fillna(method='ffill').loc[testIndex],
                  ('price','low'):tempDf.price['last'].fillna(method='ffill').loc[testIndex],
                  ('price','last'):tempDf.price['last'].fillna(method='ffill').loc[testIndex]}) 
    
def test_fill_values_2():
    # We can fill the empty elements with the previous non-empty value of the last price in the following way
    tmp = tempDf.price['last'].fillna(method='ffill').loc[testIndex]
    pd.DataFrame({('price','first'):tmp,
                  ('price','high'):tmp,
                  ('price','low'):tmp,
                  ('price','last'):tmp}) 
    
    
    
%timeit test_fill_1()
%timeit test_fill_2()

10.5 ms ± 609 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.43 ms ± 329 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [87]:
# Fill out the empty ones!
tempDf.loc[testIndex] = pd.DataFrame({('price','first'):tempDf.price['last'].fillna(method='ffill').loc[testIndex],
                                      ('price','high'):tempDf.price['last'].fillna(method='ffill').loc[testIndex],
                                      ('price','low'):tempDf.price['last'].fillna(method='ffill').loc[testIndex],
                                      ('price','last'):tempDf.price['last'].fillna(method='ffill').loc[testIndex]})

In [165]:
def test_fill_assign_1():
    # We can fill the empty elements with the previous non-empty value of the last price in the following way
    tempDf.loc[testIndex] = pd.DataFrame({('price','first'):tempDf.price['last'].fillna(method='ffill').loc[testIndex],
                                          ('price','high'):tempDf.price['last'].fillna(method='ffill').loc[testIndex],
                                          ('price','low'):tempDf.price['last'].fillna(method='ffill').loc[testIndex],
                                          ('price','last'):tempDf.price['last'].fillna(method='ffill').loc[testIndex]})
    
def test_fill_assign_2():
    # We can fill the empty elements with the previous non-empty value of the last price in the following way
    tmp = tempDf.price['last'].fillna(method='ffill').loc[testIndex]
    tempDf.loc[testIndex] = pd.DataFrame({('price','first'):tmp,
                                          ('price','high'):tmp,
                                          ('price','low'):tmp,
                                          ('price','last'):tmp}) 
    
    
%timeit test_fill_assign_1()
%timeit test_fill_assign_2()

18.4 ms ± 1.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
12.9 ms ± 850 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [166]:
len(OHLC.index.get_level_values(1))
#!=((len(time_bins)-2)*dayz):

780

In [169]:
OHLC.index.get_level_values(1)[389]

389

In [138]:
tempDf.price['last'].fillna(method='ffill').loc[testIndex]

Date      time_group
20200401  24            1120.93
          72            1126.05
          159           1115.26
          190           1106.76
          216           1111.25
          246           1109.42
          271           1112.04
          285           1110.20
          293           1109.24
          370           1097.88
20200402  12            1105.00
          16            1096.50
          83            1103.50
          106           1109.71
          133           1114.61
          176           1119.50
          178           1118.71
          199           1122.25
          229           1114.71
          375           1117.09
Name: last, dtype: float64

In [162]:
#tempDf.loc[testIndex, ('price', slice(None))] = 1
tempDf.loc[testIndex] = [np.zeros(shape=(20,1)), np.zeros(shape=(20,1)), np.zeros(shape=(20,1)), np.zeros(shape=(20,1))]
tempDf

ValueError: cannot set using a list-like indexer with a different length than the value

In [88]:
# Check that it worked - v1
tempDf.count()

price  first    780
       high     780
       low      780
       last     780
dtype: int64

In [89]:
# # Check that it worked - v2
tempDf.loc[testIndex]

price                           
                       first     high      low     last
Date     time_group                                    
20200401 24          1120.93  1120.93  1120.93  1120.93
         72          1126.05  1126.05  1126.05  1126.05
         159         1115.26  1115.26  1115.26  1115.26
         190         1106.76  1106.76  1106.76  1106.76
         216         1111.25  1111.25  1111.25  1111.25
         246         1109.42  1109.42  1109.42  1109.42
         271         1112.04  1112.04  1112.04  1112.04
         285         1110.20  1110.20  1110.20  1110.20
         293         1109.24  1109.24  1109.24  1109.24
         370         1097.88  1097.88  1097.88  1097.88
20200402 12          1105.00  1105.00  1105.00  1105.00
         16          1096.50  1096.50  1096.50  1096.50
         83          1103.50  1103.50  1103.50  1103.50
         106         1109.71  1109.71  1109.71  1109.71
         133         1114.61  1114.61  1114.61  1114.61
         176         1119.50  1119.50  1119.50  1119.50
         178         1118.71  1118.71  1118.71  1118.71
         199         1122.25  1122.25  1122.25  1122.25
         229         1114.71  1114.71  1114.71  1114.71
         375         1117.09  1117.09  1117.09  1117.09

# Lets verify that it is actually the previous close we have filled in all empty rows.

In [90]:
indeX = np.array(testIndex.get_level_values(1)-1)#.reshape((10,2))
indeX

array([ 23,  71, 158, 189, 215, 245, 270, 284, 292, 369,  11,  15,  82,
       105, 132, 175, 177, 198, 228, 374], dtype=int64)

In [91]:
secondIndeX = np.array(['20200401' if ele < len(indeX)/2 else '20200402' for ele in np.arange(len(indeX))])
secondIndeX

array(['20200401', '20200401', '20200401', '20200401', '20200401',
       '20200401', '20200401', '20200401', '20200401', '20200401',
       '20200402', '20200402', '20200402', '20200402', '20200402',
       '20200402', '20200402', '20200402', '20200402', '20200402'],
      dtype='<U8')

In [92]:
tempDf.loc[pd.MultiIndex.from_arrays([secondIndeX,
                            indeX])]

price                           
                first     high      low     last
20200401 23   1118.67  1120.93  1118.03  1120.93
         71   1126.69  1127.14  1126.01  1126.05
         158  1115.78  1116.00  1115.26  1115.26
         189  1107.58  1107.58  1106.01  1106.76
         215  1111.97  1112.82  1111.00  1111.25
         245  1108.26  1109.42  1108.26  1109.42
         270  1111.77  1112.81  1111.69  1112.04
         284  1110.35  1110.72  1109.85  1110.20
         292  1110.02  1110.02  1109.23  1109.24
         369  1098.86  1099.00  1097.44  1097.88
20200402 11   1106.29  1106.29  1103.66  1105.00
         15   1099.37  1101.00  1096.48  1096.50
         82   1106.86  1106.87  1103.50  1103.50
         105  1106.34  1109.71  1106.34  1109.71
         132  1114.84  1115.17  1113.71  1114.61
         175  1118.30  1120.00  1118.30  1119.50
         177  1119.33  1119.38  1118.00  1118.71
         198  1122.64  1122.64  1122.00  1122.25
         228  1114.84  1116.15  1114.65  1114.71
         374  1115.07  1118.00  1115.07  1117.09

# Lets implement it in a function.

In [93]:
len(time_bins)

392

In [94]:
step = 1/10
delta = step/60
time_bins = np.arange(9.5-delta
                      ,16#+delta
                      ,delta)

In [95]:
# set up as function
def candleCreateNP_vect_v2(verbose=True,createHoles=False,holes=10,fillHoles=True):
        
    cleanedData['hour_min_col'] = cleanedData['Hour'] + cleanedData['Minute']/60
    if verbose:
        print(f"min and max of new hour_min_col: \
              {cleanedData['hour_min_col'].min()}, {cleanedData['hour_min_col'].max()}")
              
    # setup time_bins to group each timestamp
    #print(step)
    delta = step/60
    time_bins = np.arange(9.5-delta, 16, delta)
              
    # put each timestamp into a bucket according to time_bins defined by the step variable
    cleanedData['time_group'] = pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=True, labels=False)
    
    # group by date and time_group, extract price, take it first, max, min, last (open, high, low, close)
    OHLC = cleanedData.groupby(['Date','time_group'])[['price']].agg(['first', 'max', 'min', 'last'])    
    #print(OHLC.index)
    # Create some holes in the candle-table, so we can verify that the method works.
    if createHoles:
        #np.random.seed(2021)
        for dt in OHLC.index.get_level_values(0).unique():
            dropedElements = []
#             print(dt,'\n')
            for inx in np.arange(holes):
                #print(len(time_bins))
                rm = np.random.randint(0,len(time_bins)-2,1,)
                dropedElements.append(rm[0])
                OHLC = OHLC.drop((dt,rm[0]))
#             print('Elements dropped: ',dropedElements)
    #print(len(OHLC.index.get_level_values(1)))
    #print((len(time_bins)-2))
    #Let check if we are missing any values
    dayz = len(OHLC.index.get_level_values(0).unique())
    if len(OHLC.index.get_level_values(1))!=((len(time_bins)-2)*dayz):
        #print('Filling')
        ##### Creating our temporary table, with all the indices that is surposed to be in the actual candle-table.
        ## Creating the multiIndex-index
        mtInd = pd.MultiIndex.from_product([OHLC.index.get_level_values(0).unique(),np.arange(len(time_bins)-2)],
                                   names=['Date','time_group'])

        ## Creating the multiIndex-columns
        mtCol = pd.MultiIndex.from_product([['price'],['first','high','low','last']])

        ## Creating the table itself
        tempDf = pd.DataFrame(np.nan
                              ,columns=mtCol
                              ,index=mtInd)
        #print(tempDf.shape)
        # Filling the non-empty elements of OHLC into the temp-table
        tempDf.loc[OHLC.index]=OHLC.copy(deep=True)

        # To see that the filling mechanism works:
        if fillHoles:

            # Storing the indices to be filled
            toBeFilled = tempDf[tempDf.price['first'].isna()].index

            # Fill out the empty ones!
            tempDf.loc[toBeFilled] = pd.DataFrame({('price','first'):tempDf.price['last'].fillna(method='ffill').loc[toBeFilled],
                                                  ('price','high'):tempDf.price['last'].fillna(method='ffill').loc[toBeFilled],
                                                  ('price','low'):tempDf.price['last'].fillna(method='ffill').loc[toBeFilled],
                                                  ('price','last'):tempDf.price['last'].fillna(method='ffill').loc[toBeFilled]})

        # Return the complete data
        return tempDf
    
    else:
        
        # return as numpy if preferred
        return OHLC#.values

In [106]:
candles = candleCreateNP_vect_v2(holes=10,createHoles=False,fillHoles=True)
candles

min and max of new hour_min_col:               9.5, 15.983333333333333


price                           
                        first      max      min     last
Date     time_group                                     
20200401 0           1122.260  1127.22  1119.51  1126.12
         1           1127.570  1128.00  1120.00  1126.92
         2           1127.190  1127.40  1124.02  1125.44
         3           1125.010  1125.19  1122.97  1123.00
         4           1123.670  1123.68  1120.01  1121.21
         5           1121.270  1121.85  1114.79  1117.44
         6           1117.040  1124.40  1117.04  1124.40
         7           1124.980  1127.99  1124.97  1127.65
         8           1127.630  1128.70  1125.18  1125.51
         9           1126.950  1127.52  1124.27  1124.52
         10          1124.720  1125.89  1123.07  1124.52
         11          1124.120  1126.00  1123.80  1125.77
         12          1125.810  1127.62  1125.35  1125.35
         13          1126.550  1127.85  1125.18  1125.18
         14          1125.830  1126.47  1124.68  1124.76
         15          1124.140  1124.98  1122.50  1122.50
         16          1122.930  1122.93  1118.26  1118.26
         17          1119.690  1120.80  1118.88  1119.07
         18          1119.560  1120.63  1119.00  1119.77
         19          1120.000  1121.81  1119.73  1120.29
         20          1120.130  1121.52  1119.70  1120.75
         21          1121.680  1121.68  1119.98  1120.19
         22          1120.190  1120.30  1117.34  1117.61
         23          1118.670  1120.93  1118.03  1120.93
         24          1121.310  1121.61  1118.50  1118.92
         25          1118.050  1118.92  1117.70  1118.57
         26          1118.710  1119.76  1117.60  1118.55
         27          1118.750  1118.80  1116.46  1118.66
         28          1118.120  1118.12  1115.30  1115.30
         29          1115.500  1116.98  1115.18  1115.55
...                       ...      ...      ...      ...
20200402 360         1114.860  1117.09  1114.86  1115.41
         361         1114.600  1114.60  1110.56  1112.52
         362         1112.840  1114.34  1112.83  1112.83
         363         1113.320  1114.20  1112.47  1113.80
         364         1113.300  1114.20  1112.36  1112.52
         365         1112.500  1114.57  1112.50  1113.45
         366         1113.820  1115.44  1112.35  1112.35
         367         1112.360  1112.36  1110.43  1111.08
         368         1111.580  1112.42  1111.13  1112.21
         369         1111.610  1113.34  1111.61  1112.42
         370         1112.700  1115.34  1112.70  1114.36
         371         1114.360  1116.00  1112.71  1115.06
         372         1115.670  1117.39  1113.93  1114.38
         373         1113.940  1117.46  1113.50  1115.60
         374         1115.070  1118.00  1115.07  1117.09
         375         1116.560  1118.60  1116.23  1118.21
         376         1117.690  1117.69  1115.00  1115.28
         377         1115.310  1116.41  1113.00  1113.50
         378         1113.770  1114.24  1112.79  1113.53
         379         1113.790  1114.12  1111.00  1114.12
         380         1114.940  1121.08  1114.94  1119.84
         381         1120.400  1120.40  1116.59  1117.08
         382         1117.290  1118.25  1115.66  1116.62
         383         1117.050  1118.24  1114.64  1114.80
         384         1115.870  1123.67  1115.72  1120.98
         385         1119.880  1122.88  1117.00  1117.24
         386         1116.450  1122.96  1112.18  1121.41
         387         1121.430  1122.86  1119.76  1121.95
         388         1122.580  1123.60  1121.18  1122.86
         389         1122.725  1124.00  1119.64  1121.74

[780 rows x 4 columns]

In [107]:
candles.count()

price  first    780
       max      780
       min      780
       last     780
dtype: int64

In [108]:
candles.shape

(780, 4)

In [109]:
candles[candles.price['first'].isna()]

Empty DataFrame
Columns: [(price, first), (price, max), (price, min), (price, last)]
Index: []

## Let verify that the filling mechanism works

In [110]:
candlesFilled = candleCreateNP_vect_v2(holes=10,createHoles=True,fillHoles=True)
#candlesFilled

min and max of new hour_min_col:               9.5, 15.983333333333333


In [111]:
candlesFilled[candles.price['first'].isna()]

Empty DataFrame
Columns: [(price, first), (price, high), (price, low), (price, last)]
Index: []

# Lets time it

In [119]:
step = 1 #1/10
# Aggregate data in candle sticks
%timeit candleNP1 = candleCreateNP_vect(verbose=False)
%timeit candleNP2 = candleCreateNP_vect_v2(verbose=False)

21.2 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
19.6 ms ± 999 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [120]:
candleCreateNP_vect(verbose=False).shape

(780, 4)

In [121]:
candleCreateNP_vect_v2(verbose=False).shape

(780, 4)

In [122]:
(candleCreateNP_vect(verbose=False).values == candleCreateNP_vect_v2(verbose=False).values).all()

True

In [123]:
candleCreateNP_vect(verbose=False)

price                           
                        first      max      min     last
Date     time_group                                     
20200401 0           1122.260  1127.22  1119.51  1126.12
         1           1127.570  1128.00  1120.00  1126.92
         2           1127.190  1127.40  1124.02  1125.44
         3           1125.010  1125.19  1122.97  1123.00
         4           1123.670  1123.68  1120.01  1121.21
         5           1121.270  1121.85  1114.79  1117.44
         6           1117.040  1124.40  1117.04  1124.40
         7           1124.980  1127.99  1124.97  1127.65
         8           1127.630  1128.70  1125.18  1125.51
         9           1126.950  1127.52  1124.27  1124.52
         10          1124.720  1125.89  1123.07  1124.52
         11          1124.120  1126.00  1123.80  1125.77
         12          1125.810  1127.62  1125.35  1125.35
         13          1126.550  1127.85  1125.18  1125.18
         14          1125.830  1126.47  1124.68  1124.76
         15          1124.140  1124.98  1122.50  1122.50
         16          1122.930  1122.93  1118.26  1118.26
         17          1119.690  1120.80  1118.88  1119.07
         18          1119.560  1120.63  1119.00  1119.77
         19          1120.000  1121.81  1119.73  1120.29
         20          1120.130  1121.52  1119.70  1120.75
         21          1121.680  1121.68  1119.98  1120.19
         22          1120.190  1120.30  1117.34  1117.61
         23          1118.670  1120.93  1118.03  1120.93
         24          1121.310  1121.61  1118.50  1118.92
         25          1118.050  1118.92  1117.70  1118.57
         26          1118.710  1119.76  1117.60  1118.55
         27          1118.750  1118.80  1116.46  1118.66
         28          1118.120  1118.12  1115.30  1115.30
         29          1115.500  1116.98  1115.18  1115.55
...                       ...      ...      ...      ...
20200402 360         1114.860  1117.09  1114.86  1115.41
         361         1114.600  1114.60  1110.56  1112.52
         362         1112.840  1114.34  1112.83  1112.83
         363         1113.320  1114.20  1112.47  1113.80
         364         1113.300  1114.20  1112.36  1112.52
         365         1112.500  1114.57  1112.50  1113.45
         366         1113.820  1115.44  1112.35  1112.35
         367         1112.360  1112.36  1110.43  1111.08
         368         1111.580  1112.42  1111.13  1112.21
         369         1111.610  1113.34  1111.61  1112.42
         370         1112.700  1115.34  1112.70  1114.36
         371         1114.360  1116.00  1112.71  1115.06
         372         1115.670  1117.39  1113.93  1114.38
         373         1113.940  1117.46  1113.50  1115.60
         374         1115.070  1118.00  1115.07  1117.09
         375         1116.560  1118.60  1116.23  1118.21
         376         1117.690  1117.69  1115.00  1115.28
         377         1115.310  1116.41  1113.00  1113.50
         378         1113.770  1114.24  1112.79  1113.53
         379         1113.790  1114.12  1111.00  1114.12
         380         1114.940  1121.08  1114.94  1119.84
         381         1120.400  1120.40  1116.59  1117.08
         382         1117.290  1118.25  1115.66  1116.62
         383         1117.050  1118.24  1114.64  1114.80
         384         1115.870  1123.67  1115.72  1120.98
         385         1119.880  1122.88  1117.00  1117.24
         386         1116.450  1122.96  1112.18  1121.41
         387         1121.430  1122.86  1119.76  1121.95
         388         1122.580  1123.60  1121.18  1122.86
         389         1122.725  1124.00  1119.64  1121.74

[780 rows x 4 columns]

In [124]:
candleCreateNP_vect_v2(verbose=False)

price                           
                        first      max      min     last
Date     time_group                                     
20200401 0           1122.260  1127.22  1119.51  1126.12
         1           1127.570  1128.00  1120.00  1126.92
         2           1127.190  1127.40  1124.02  1125.44
         3           1125.010  1125.19  1122.97  1123.00
         4           1123.670  1123.68  1120.01  1121.21
         5           1121.270  1121.85  1114.79  1117.44
         6           1117.040  1124.40  1117.04  1124.40
         7           1124.980  1127.99  1124.97  1127.65
         8           1127.630  1128.70  1125.18  1125.51
         9           1126.950  1127.52  1124.27  1124.52
         10          1124.720  1125.89  1123.07  1124.52
         11          1124.120  1126.00  1123.80  1125.77
         12          1125.810  1127.62  1125.35  1125.35
         13          1126.550  1127.85  1125.18  1125.18
         14          1125.830  1126.47  1124.68  1124.76
         15          1124.140  1124.98  1122.50  1122.50
         16          1122.930  1122.93  1118.26  1118.26
         17          1119.690  1120.80  1118.88  1119.07
         18          1119.560  1120.63  1119.00  1119.77
         19          1120.000  1121.81  1119.73  1120.29
         20          1120.130  1121.52  1119.70  1120.75
         21          1121.680  1121.68  1119.98  1120.19
         22          1120.190  1120.30  1117.34  1117.61
         23          1118.670  1120.93  1118.03  1120.93
         24          1121.310  1121.61  1118.50  1118.92
         25          1118.050  1118.92  1117.70  1118.57
         26          1118.710  1119.76  1117.60  1118.55
         27          1118.750  1118.80  1116.46  1118.66
         28          1118.120  1118.12  1115.30  1115.30
         29          1115.500  1116.98  1115.18  1115.55
...                       ...      ...      ...      ...
20200402 360         1114.860  1117.09  1114.86  1115.41
         361         1114.600  1114.60  1110.56  1112.52
         362         1112.840  1114.34  1112.83  1112.83
         363         1113.320  1114.20  1112.47  1113.80
         364         1113.300  1114.20  1112.36  1112.52
         365         1112.500  1114.57  1112.50  1113.45
         366         1113.820  1115.44  1112.35  1112.35
         367         1112.360  1112.36  1110.43  1111.08
         368         1111.580  1112.42  1111.13  1112.21
         369         1111.610  1113.34  1111.61  1112.42
         370         1112.700  1115.34  1112.70  1114.36
         371         1114.360  1116.00  1112.71  1115.06
         372         1115.670  1117.39  1113.93  1114.38
         373         1113.940  1117.46  1113.50  1115.60
         374         1115.070  1118.00  1115.07  1117.09
         375         1116.560  1118.60  1116.23  1118.21
         376         1117.690  1117.69  1115.00  1115.28
         377         1115.310  1116.41  1113.00  1113.50
         378         1113.770  1114.24  1112.79  1113.53
         379         1113.790  1114.12  1111.00  1114.12
         380         1114.940  1121.08  1114.94  1119.84
         381         1120.400  1120.40  1116.59  1117.08
         382         1117.290  1118.25  1115.66  1116.62
         383         1117.050  1118.24  1114.64  1114.80
         384         1115.870  1123.67  1115.72  1120.98
         385         1119.880  1122.88  1117.00  1117.24
         386         1116.450  1122.96  1112.18  1121.41
         387         1121.430  1122.86  1119.76  1121.95
         388         1122.580  1123.60  1121.18  1122.86
         389         1122.725  1124.00  1119.64  1121.74

[780 rows x 4 columns]

# Add function to produce smaller candles than 1 min

In [280]:
# Final vectorized function
def candleCreateNP_vect_v5(data
                        ,step
                        ,verbose=False
                        ,fillHoles=True):

    # v1-v4:
    #data['hour_min_col'] = data['Hour'] + data['Minute']/60
    
    # v5:
    Timestamp_dt = data['Timestamp'].dt
    Timestamp_float = Timestamp_dt.hour \
                      + Timestamp_dt.minute/60 \
                      + Timestamp_dt.second/(60*60) \
                      + Timestamp_dt.microsecond/(60*60*10**6)
    data['hour_min_col'] = Timestamp_float    
    
    if verbose:
        print(f"min and max of new hour_min_col: \
              {data['hour_min_col'].min()}, {data['hour_min_col'].max()}")

    # setup time_bins to group each timestamp
    delta = step/60
    time_bins = np.arange(9.5, 16+delta, delta) #np.arange(9.5-delta, 16, delta)

    # put each timestamp into a bucket according to time_bins defined by the step variable
    data['time_group'] = pd.cut(data['hour_min_col'], bins=time_bins, right=True, labels=False)

    # group by date and time_group, extract price, take it open, max, min, last (open, high, low, close)
    OHLC = data.groupby(['Date','time_group'])[['price']].agg(['first', 'max', 'min', 'last'])
    OHLC = OHLC.rename(columns={'first':'open'
                                ,'max':'high'
                                ,'min':'low'
                                ,'last':'close'})
              
    #Let check if we are missing any values
    dayz = len(OHLC.index.get_level_values(0).unique())
    if len(OHLC.index.get_level_values(1))!=((len(time_bins)-1)*dayz):

        ##### Creating our temporary table, with all the indices that is surposed to be in the actual candle-table.
        ## Creating the multiIndex-index
        mtInd = pd.MultiIndex.from_product([OHLC.index.get_level_values(0).unique(),np.arange(len(time_bins)-1)],
                                   names=['Date','time_group'])

        ## Creating the multiIndex-columns
        mtCol = pd.MultiIndex.from_product([['price'],['open','high','low','close']])

        ## Creating the table itself
        tempDf = pd.DataFrame(np.nan
                              ,columns=mtCol
                              ,index=mtInd)

        # Filling the non-empty elements of OHLC into the temp-table
        tempDf.loc[OHLC.index]=OHLC.copy(deep=True)

        # To see that the filling mechanism works:
        if fillHoles:

            # Storing the indices to be filled
            toBeFilled = tempDf[tempDf.price['open'].isna()].index

            # Fill out the empty ones!
            dataToFillIn = tempDf.price['close'].fillna(method='ffill').loc[toBeFilled]
            tempDf.loc[toBeFilled] = pd.DataFrame({('price','open'):dataToFillIn,
                                                  ('price','high'):dataToFillIn,
                                                  ('price','low'):dataToFillIn,
                                                  ('price','close'):dataToFillIn})

        # Return the complete data
        return tempDf

    else:

        # return as numpy if preferred
        return OHLC.values

In [291]:
data = cleanedData.copy()
step = .1
Timestamp_dt = data['Timestamp'].dt
Timestamp_float = Timestamp_dt.hour \
                  + Timestamp_dt.minute/60 \
                  + Timestamp_dt.second/(60*60) \
                  + Timestamp_dt.microsecond/(60*60*10**6)
data['hour_min_col'] = Timestamp_float    

if verbose:
    print(f"min and max of new hour_min_col: \
          {data['hour_min_col'].min()}, {data['hour_min_col'].max()}")

# setup time_bins to group each timestamp
delta = step/60
time_bins = np.arange(9.5, 16+delta, delta) #np.arange(9.5-delta, 16, delta)

# put each timestamp into a bucket according to time_bins defined by the step variable
data['time_group'] = pd.cut(data['hour_min_col'], bins=time_bins, right=True, labels=False)

# group by date and time_group, extract price, take it open, max, min, last (open, high, low, close)
OHLC = data.groupby(['Date','time_group'])[['price']].agg(['first', 'max', 'min', 'last'])
OHLC = OHLC.rename(columns={'first':'open'
                            ,'max':'high'
                            ,'min':'low'
                            ,'last':'close'})
OHLC
# #Let check if we are missing any values
dayz = len(OHLC.index.get_level_values(0).unique())
# dayz
# len(OHLC.index.get_level_values(1))
# ((len(time_bins)-1)*dayz)
if len(OHLC.index.get_level_values(1))!=((len(time_bins)-1)*dayz):

    ##### Creating our temporary table, with all the indices that is surposed to be in the actual candle-table.
    ## Creating the multiIndex-index
    mtInd = pd.MultiIndex.from_product([OHLC.index.get_level_values(0).unique(),np.arange(len(time_bins)-1)],
                               names=['Date','time_group'])

    ## Creating the multiIndex-columns
    mtCol = pd.MultiIndex.from_product([['price'],['open','high','low','close']])

    ## Creating the table itself
    tempDf = pd.DataFrame(np.nan
                          ,columns=mtCol
                          ,index=mtInd)

    # Filling the non-empty elements of OHLC into the temp-table
    tempDf.loc[OHLC.index]=OHLC.copy(deep=True)

    # To see that the filling mechanism works:
    if fillHoles:

        # Storing the indices to be filled
        toBeFilled = tempDf[tempDf.price['open'].isna()].index

        # Fill out the empty ones!
        dataToFillIn = tempDf.price['close'].fillna(method='ffill').loc[toBeFilled]
        tempDf.loc[toBeFilled] = pd.DataFrame({('price','open'):dataToFillIn,
                                              ('price','high'):dataToFillIn,
                                              ('price','low'):dataToFillIn,
                                              ('price','close'):dataToFillIn})


min and max of new hour_min_col:           9.500003529444445, 15.999999836944443


price                            
                         open      high      low    close
Date     time_group                                      
20200401 0           1122.260  1127.220  1121.00  1126.02
         1           1126.120  1126.120  1124.97  1124.97
         2           1124.880  1126.170  1120.88  1121.35
         3           1121.350  1123.570  1120.66  1121.34
         4           1122.370  1122.520  1121.01  1122.52
         5           1121.510  1121.875  1121.20  1121.20
         6           1121.885  1122.370  1121.88  1122.20
         7           1121.590  1121.730  1119.51  1121.72
         8           1121.720  1123.050  1121.72  1122.78
         9           1122.500  1126.120  1122.48  1126.12
         10          1127.570  1128.000  1125.43  1125.43
         12          1125.000  1125.000  1121.55  1122.47
         13          1122.480  1122.480  1122.00  1122.00
         14          1122.210  1122.210  1120.00  1120.42
         15          1122.000  1124.690  1122.00  1124.69
         16          1122.900  1123.340  1122.12  1122.12
         17          1124.360  1125.000  1124.32  1124.38
         18          1124.870  1127.340  1123.78  1127.34
         19          1126.650  1126.920  1125.93  1126.92
         20          1127.190  1127.190  1127.19  1127.19
         21          1126.440  1126.500  1126.44  1126.50
         22          1125.930  1126.250  1125.56  1125.56
         23          1125.550  1125.550  1124.09  1124.49
         24          1124.490  1124.950  1124.16  1124.73
         25          1124.500  1124.500  1124.50  1124.50
         26          1124.290  1124.780  1124.08  1124.08
         27          1124.070  1126.450  1124.02  1126.33
         28          1125.520  1127.400  1125.07  1126.76
         29          1126.020  1126.020  1125.44  1125.44
         30          1125.010  1125.010  1122.97  1123.65
...                       ...       ...      ...      ...
20200402 3870        1121.430  1121.430  1119.83  1119.83
         3871        1119.900  1121.700  1119.87  1120.34
         3872        1120.000  1120.430  1119.76  1120.43
         3873        1119.790  1121.910  1119.78  1121.40
         3874        1121.000  1121.000  1121.00  1121.00
         3875        1120.670  1122.250  1120.67  1122.01
         3876        1122.190  1122.760  1121.69  1121.69
         3877        1121.690  1122.000  1121.00  1121.00
         3878        1120.430  1122.670  1120.43  1122.67
         3879        1122.840  1122.860  1121.95  1121.95
         3880        1122.580  1122.580  1121.69  1122.39
         3881        1121.610  1122.480  1121.36  1122.32
         3882        1121.660  1122.140  1121.18  1122.10
         3883        1121.850  1122.750  1121.41  1122.75
         3884        1122.750  1122.940  1122.25  1122.92
         3885        1122.330  1123.600  1121.56  1122.00
         3886        1121.830  1122.480  1121.83  1122.48
         3887        1121.890  1122.790  1121.89  1122.79
         3888        1122.520  1122.900  1122.05  1122.90
         3889        1122.580  1122.860  1122.22  1122.86
         3890        1122.725  1122.850  1120.69  1120.69
         3891        1121.520  1122.090  1120.85  1121.36
         3892        1122.340  1123.230  1122.11  1122.83
         3893        1122.610  1122.610  1120.10  1120.10
         3894        1120.440  1120.950  1119.64  1120.39
         3895        1120.220  1121.330  1119.98  1121.33
         3896        1120.930  1122.420  1120.93  1122.00
         3897        1121.650  1122.990  1121.03  1122.92
         3898        1122.930  1124.000  1122.08  1122.19
         3899        1123.120  1123.170  1120.44  1121.74

[5112 rows x 4 columns]

In [285]:
np.arange(len(time_bins))

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [288]:
time_bins

array([ 9.5       ,  9.51666667,  9.53333333,  9.55      ,  9.56666667,
        9.58333333,  9.6       ,  9.61666667,  9.63333333,  9.65      ,
        9.66666667,  9.68333333,  9.7       ,  9.71666667,  9.73333333,
        9.75      ,  9.76666667,  9.78333333,  9.8       ,  9.81666667,
        9.83333333,  9.85      ,  9.86666667,  9.88333333,  9.9       ,
        9.91666667,  9.93333333,  9.95      ,  9.96666667,  9.98333333,
       10.        , 10.01666667, 10.03333333, 10.05      , 10.06666667,
       10.08333333, 10.1       , 10.11666667, 10.13333333, 10.15      ,
       10.16666667, 10.18333333, 10.2       , 10.21666667, 10.23333333,
       10.25      , 10.26666667, 10.28333333, 10.3       , 10.31666667,
       10.33333333, 10.35      , 10.36666667, 10.38333333, 10.4       ,
       10.41666667, 10.43333333, 10.45      , 10.46666667, 10.48333333,
       10.5       , 10.51666667, 10.53333333, 10.55      , 10.56666667,
       10.58333333, 10.6       , 10.61666667, 10.63333333, 10.65

In [308]:
test_v5 = candleCreateNP_vect_v5(data=cleanedData, step=.1, verbose=True, fillHoles=True)
test_v5

min and max of new hour_min_col:               9.500003529444445, 15.999999836944443


price                            
                         open      high      low    close
Date     time_group                                      
20200401 0           1122.260  1127.220  1121.00  1126.02
         1           1126.120  1126.120  1124.97  1124.97
         2           1124.880  1126.170  1120.88  1121.35
         3           1121.350  1123.570  1120.66  1121.34
         4           1122.370  1122.520  1121.01  1122.52
         5           1121.510  1121.875  1121.20  1121.20
         6           1121.885  1122.370  1121.88  1122.20
         7           1121.590  1121.730  1119.51  1121.72
         8           1121.720  1123.050  1121.72  1122.78
         9           1122.500  1126.120  1122.48  1126.12
         10          1127.570  1128.000  1125.43  1125.43
         11          1125.430  1125.430  1125.43  1125.43
         12          1125.000  1125.000  1121.55  1122.47
         13          1122.480  1122.480  1122.00  1122.00
         14          1122.210  1122.210  1120.00  1120.42
         15          1122.000  1124.690  1122.00  1124.69
         16          1122.900  1123.340  1122.12  1122.12
         17          1124.360  1125.000  1124.32  1124.38
         18          1124.870  1127.340  1123.78  1127.34
         19          1126.650  1126.920  1125.93  1126.92
         20          1127.190  1127.190  1127.19  1127.19
         21          1126.440  1126.500  1126.44  1126.50
         22          1125.930  1126.250  1125.56  1125.56
         23          1125.550  1125.550  1124.09  1124.49
         24          1124.490  1124.950  1124.16  1124.73
         25          1124.500  1124.500  1124.50  1124.50
         26          1124.290  1124.780  1124.08  1124.08
         27          1124.070  1126.450  1124.02  1126.33
         28          1125.520  1127.400  1125.07  1126.76
         29          1126.020  1126.020  1125.44  1125.44
...                       ...       ...      ...      ...
20200402 3870        1121.430  1121.430  1119.83  1119.83
         3871        1119.900  1121.700  1119.87  1120.34
         3872        1120.000  1120.430  1119.76  1120.43
         3873        1119.790  1121.910  1119.78  1121.40
         3874        1121.000  1121.000  1121.00  1121.00
         3875        1120.670  1122.250  1120.67  1122.01
         3876        1122.190  1122.760  1121.69  1121.69
         3877        1121.690  1122.000  1121.00  1121.00
         3878        1120.430  1122.670  1120.43  1122.67
         3879        1122.840  1122.860  1121.95  1121.95
         3880        1122.580  1122.580  1121.69  1122.39
         3881        1121.610  1122.480  1121.36  1122.32
         3882        1121.660  1122.140  1121.18  1122.10
         3883        1121.850  1122.750  1121.41  1122.75
         3884        1122.750  1122.940  1122.25  1122.92
         3885        1122.330  1123.600  1121.56  1122.00
         3886        1121.830  1122.480  1121.83  1122.48
         3887        1121.890  1122.790  1121.89  1122.79
         3888        1122.520  1122.900  1122.05  1122.90
         3889        1122.580  1122.860  1122.22  1122.86
         3890        1122.725  1122.850  1120.69  1120.69
         3891        1121.520  1122.090  1120.85  1121.36
         3892        1122.340  1123.230  1122.11  1122.83
         3893        1122.610  1122.610  1120.10  1120.10
         3894        1120.440  1120.950  1119.64  1120.39
         3895        1120.220  1121.330  1119.98  1121.33
         3896        1120.930  1122.420  1120.93  1122.00
         3897        1121.650  1122.990  1121.03  1122.92
         3898        1122.930  1124.000  1122.08  1122.19
         3899        1123.120  1123.170  1120.44  1121.74

[7800 rows x 4 columns]

In [305]:
test_v5.isnull().sum()

price  open     2688
       high     2688
       low      2688
       close    2688
dtype: int64